In [13]:
using CSV
using Dates
using DataFramesMeta

include("../data/datacatalog.jl");

In [14]:
dtFmt = "yyyy-mm-ddTHH:MM:SS.s"

"yyyy-mm-ddTHH:MM:SS.s"

In [15]:
analysisYear = 2024
batteryDuration = "8hr"

"8hr"

In [24]:
dispatch = CSV.read("$batteryDuration-$analysisYear-cap-bonus-battery-optimization-results.csv", DataFrame; dateformat="yyyy-mm-ddTHH:MM:SS.s");
raHours = CSV.read("../Data/MISO RA Hours PY25.csv", DataFrame; dateformat="yyyy-mm-ddTHH:MM:SS.s");
raHours = @chain raHours begin
    @rtransform :year = year(:interval_start_est)
    @rsubset :year == analysisYear
end;
raHoursBehavior = leftjoin(raHours, dispatch, on=:interval_start_est);

In [25]:
@rsubset raHoursBehavior begin
    :charging > 0.09
    :ra_hour_north_central == 1
end

Row,interval_start_est,season,ra_hour_north_central,ra_hour_south,year,minnHubLmp,charging,discharging,SOCstart,SOCend,cashFlow
,DateTime,String7,Int64,Int64,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,2024-01-14T20:00:00,Winter,1,0,2024,104.99,3.52941,0.0,50.0,53.5294,-370.553
2,2024-01-14T21:00:00,Winter,1,0,2024,97.76,10.0,0.0,53.0,63.0,-977.6
3,2024-01-14T22:00:00,Winter,1,0,2024,93.76,10.0,0.0,61.5,71.5,-937.6
4,2024-01-15T12:00:00,Winter,1,0,2024,133.45,10.0,0.0,19.9949,29.9949,-1334.5
5,2024-01-15T22:00:00,Winter,1,0,2024,134.03,2.94713,0.0,22.4949,25.4421,-395.004
6,2024-01-15T23:00:00,Winter,1,0,2024,108.39,10.0,0.0,25.0,35.0,-1083.9
7,2024-05-21T13:00:00,Spring,1,0,2024,22.37,10.0,0.0,4.0,14.0,-223.7
8,2024-05-21T14:00:00,Spring,1,0,2024,22.4,8.82353,0.0,12.5,21.3235,-197.647


In [26]:
@rsubset raHoursBehavior begin
    :discharging > 0.09
    :ra_hour_north_central == 1
end

Row,interval_start_est,season,ra_hour_north_central,ra_hour_south,year,minnHubLmp,charging,discharging,SOCstart,SOCend,cashFlow
,DateTime,String7,Int64,Int64,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,2024-01-14T18:00:00,Winter,1,0,2024,146.65,0.0,10.0,70.0,60.0,1466.5
2,2024-01-14T19:00:00,Winter,1,0,2024,124.82,0.0,10.0,60.0,50.0,1248.2
3,2024-01-15T07:00:00,Winter,1,0,2024,185.55,0.0,10.0,69.9949,59.9949,1855.5
4,2024-01-15T08:00:00,Winter,1,0,2024,199.03,0.0,10.0,59.9949,49.9949,1990.3
5,2024-01-15T09:00:00,Winter,1,0,2024,190.24,0.0,10.0,49.9949,39.9949,1902.4
6,2024-01-15T10:00:00,Winter,1,0,2024,169.93,0.0,10.0,39.9949,29.9949,1699.3
7,2024-01-15T11:00:00,Winter,1,0,2024,169.86,0.0,10.0,29.9949,19.9949,1698.6
8,2024-01-15T17:00:00,Winter,1,0,2024,162.12,0.0,10.0,62.4949,52.4949,1621.2
9,2024-01-15T18:00:00,Winter,1,0,2024,180.0,0.0,10.0,52.4949,42.4949,1800.0


In [7]:
@rsubset raHoursBehavior begin
    :charging <= 0.09
    :discharging <= 0.09
end

Row,interval_start_est,season,ra_hour_north_central,ra_hour_south,year,minnHubLmp,charging,discharging,SOCstart,SOCend,cashFlow
,DateTime,String7,Int64,Int64,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,2024-01-01T00:00:00,Winter,0,0,2024,24.82,0.0,0.0,0.0,0.0,0.0
2,2024-01-01T02:00:00,Winter,0,0,2024,24.82,0.0,0.0,8.5,8.5,0.0
3,2024-01-01T03:00:00,Winter,0,0,2024,24.78,0.0,0.0,8.5,8.5,0.0
4,2024-01-01T04:00:00,Winter,0,0,2024,25.01,0.0,0.0,8.5,8.5,0.0
5,2024-01-01T05:00:00,Winter,0,0,2024,24.63,0.0,0.0,8.5,8.5,0.0
6,2024-01-01T06:00:00,Winter,0,0,2024,26.09,0.0,0.0,8.5,8.5,0.0
7,2024-01-01T07:00:00,Winter,0,0,2024,28.01,0.0,0.0,8.5,8.5,0.0
8,2024-01-01T09:00:00,Winter,0,0,2024,25.53,0.0,0.0,0.0,0.0,0.0
9,2024-01-01T10:00:00,Winter,0,0,2024,25.26,0.0,0.0,0.0,0.0,0.0


In [8]:
raHoursAll = rightjoin(raHours, dispatch, on=:interval_start_est);

In [9]:
subsetPeriod = @chain raHoursAll begin
    @rsubset  begin
        :interval_start_est >= DateTime("2024-01-12T00:00:00", dtFmt)
        :interval_start_est <= DateTime("2024-01-30T00:00:00", dtFmt)
    end
    @orderby :interval_start_est
end

Row,interval_start_est,season,ra_hour_north_central,ra_hour_south,year,minnHubLmp,charging,discharging,SOCstart,SOCend,cashFlow
,DateTime,String7?,Int64?,Int64?,Int64?,Float64,Float64,Float64,Float64,Float64,Float64
1,2024-01-12T00:00:00,Winter,0,0,2024,30.73,8.82353,0.0,0.0,8.82353,-271.147
2,2024-01-12T01:00:00,Winter,0,0,2024,28.03,10.0,0.0,7.5,17.5,-280.3
3,2024-01-12T02:00:00,Winter,0,0,2024,27.59,10.0,0.0,16.0,26.0,-275.9
4,2024-01-12T03:00:00,Winter,0,0,2024,26.45,10.0,0.0,24.5,34.5,-264.5
5,2024-01-12T04:00:00,Winter,0,0,2024,26.7,10.0,0.0,33.0,43.0,-267.0
6,2024-01-12T05:00:00,Winter,0,0,2024,28.38,10.0,0.0,41.5,51.5,-283.8
7,2024-01-12T06:00:00,Winter,0,0,2024,32.89,0.0,0.0,50.0,50.0,0.0
8,2024-01-12T07:00:00,Winter,0,0,2024,41.56,0.0,10.0,50.0,40.0,415.6
9,2024-01-12T08:00:00,Winter,0,0,2024,42.22,0.0,10.0,40.0,30.0,422.2


In [10]:
CSV.write("storage-ra-hours.csv", subsetPeriod)

"storage-ra-hours.csv"

In [11]:
(20 * 365 * 10) / (65*4)

280.7692307692308